In [2]:
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import numpy as np
    import pandas as pd
    from scipy.spatial import cKDTree
    from math import sqrt, ceil
    from scipy.stats import linregress
    
    import libraries as lib
    import math 
    

In [3]:
#Lees de GPS data in

# Helper function to convert NMEA latitude/longitude to decimal degrees
def convert_to_decimal_degrees(value, direction):
    degrees = int(value) // 100
    minutes = value - (degrees * 100)
    decimal = degrees + (minutes / 60)
    
    if direction in ['S', 'W']:
        decimal = -decimal
    
    return decimal

# Function to read GPS data from a text file and convert to a DataFrame
def lees_gps_data(filepath):
    """Read GPS data from a text file and convert to a DataFrame."""
    timestamps = []
    latitudes = []
    lat_directions = []
    longitudes = []
    lon_directions = []
    hoogtes = []
    speeds = []
    
    with open(filepath, 'r') as file:
        for regel in file:
            parts = regel.split(',')
            if regel.startswith('$GPRMC'):
                # Extract time and speed from GPRMC sentence
                if len(parts) > 7:
                    timestamp = parts[1]
                    speed = parts[7]  # Speed in knots
                    timestamps.append(timestamp)
                    try:
                        speed_kmh = (float(speed) * 1.852) / 3.6  # Convert knots to m/s
                        speeds.append(speed_kmh)
                    except ValueError:
                        speeds.append(None)

            elif regel.startswith('$GPGGA'):
                # Extract latitude, longitude, and altitude from GPGGA sentence
                if len(parts) > 9:
                    latitude = parts[2]
                    lat_direction = parts[3]
                    longitude = parts[4] 
                    lon_direction = parts[5]
                    altitude = parts[9]
                    
                    # Append raw values for conversion later
                    try:
                        latitude = float(latitude)
                        latitudes.append(latitude)
                        lat_directions.append(lat_direction)
                    except ValueError:
                        latitudes.append(None)
                        lat_directions.append(None)
                    
                    try:
                        longitude = float(longitude)
                        longitudes.append(longitude)
                        lon_directions.append(lon_direction)
                    except ValueError:
                        longitudes.append(None)
                        lon_directions.append(None)
                    
                    # Convert altitude to float
                    try:
                        altitude = float(altitude)
                        hoogtes.append(altitude)
                    except ValueError:
                        hoogtes.append(None)
    
    # Check if all lists have the same length
    max_length = max(len(timestamps), len(latitudes), len(longitudes), len(hoogtes), len(speeds))
    
    # Make sure all lists have the same length by appending None to the shorter ones
    timestamps.extend([None] * (max_length - len(timestamps)))
    latitudes.extend([None] * (max_length - len(latitudes)))
    lat_directions.extend([None] * (max_length - len(lat_directions)))
    longitudes.extend([None] * (max_length - len(longitudes)))
    lon_directions.extend([None] * (max_length - len(lon_directions)))
    hoogtes.extend([None] * (max_length - len(hoogtes)))
    speeds.extend([None] * (max_length - len(speeds)))
    
    # Construct DataFrame
    data = {
        'timestamp': timestamps,
        'latitude': latitudes,
        'lat_direction': lat_directions,
        'longitude': longitudes,
        'lon_direction': lon_directions,
        'hoogte': hoogtes,
        'speed_m/s': speeds
    }
    
    df = pd.DataFrame(data)
    
    # Filter out rows where latitude or longitude are missing
    df = df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)
    
    # Convert latitude and longitude columns to decimal degrees
    df['latitude'] = df.apply(lambda row: convert_to_decimal_degrees(float(row['latitude']), row['lat_direction']), axis=1)
    df['longitude'] = df.apply(lambda row: convert_to_decimal_degrees(float(row['longitude']), row['lon_direction']), axis=1)
    
    return df

# Set file path
file_path = "C:/Users/qvled/OneDrive - HvA/Clean_Mobility_Files/H2A/Data voor hoogteprofiel.txt"

# Read GPS data
df = lees_gps_data(file_path)

# Print the resulting DataFrame
df


,timestamp,latitude,lat_direction,longitude,lon_direction,hoogte,speed_m/s
0,063448.30,43.771393,N,-0.042098,W,99.5,0.038583
1,063448.40,43.771393,N,-0.042098,W,99.5,0.026237
2,063448.50,43.771393,N,-0.042098,W,99.4,0.035497
3,063448.60,43.771393,N,-0.042098,W,99.5,0.073051
4,063448.70,43.771393,N,-0.042098,W,99.4,0.064820
...,...,...,...,...,...,...,...
46568,None,43.771202,N,-0.041941,W,96.0,NaN
46569,None,43.771202,N,-0.041941,W,96.0,NaN
46570,None,43.771202,N,-0.041941,W,96.1,NaN
46571,None,43.771202,N,-0.041941,W,96.1,NaN


In [4]:
#Functies voor het filter

def calculate_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)

def find_nearest_index(df, target_lon, target_lat):
    distances = df.apply(lambda row: calculate_distance(row['latitude'], row['longitude'], target_lat, target_lon), axis=1)
    return distances.idxmin()

def filter_track_data(df, start_lon, start_lat, end_lon, end_lat):
    start_index = find_nearest_index(df, start_lon, start_lat)
    end_index = find_nearest_index(df, end_lon, end_lat)
    
    # Zorg ervoor dat de start_index voor de end_index komt
    if start_index > end_index:
        start_index, end_index = end_index, start_index
    
    return df.iloc[start_index:end_index + 1]


In [5]:
#Filter vanaf het moment dat de auto de baan opgaat en wanneer die er weer afgaat.

# Definieer de coördinaten voor het start- en eindpunt
start_lon = -0.0446
start_lat = 0.00095 + 4.3771e1
end_lon = -0.039925
end_lat = 0.000300 + 4.3771e1

# Veronderstel dat 'df' je DataFrame is met de relevante data
filtered_df = filter_track_data(df, start_lon, start_lat, end_lon, end_lat)

# Print het gefilterde DataFrame om te controleren
filtered_df
#43.767914, -0.035855

,timestamp,latitude,lat_direction,longitude,lon_direction,hoogte,speed_m/s
22340,071203.60,43.771951,N,-0.044597,W,102.0,5.597156
22341,071203.70,43.771954,N,-0.044603,W,102.0,5.624421
22342,071203.80,43.771955,N,-0.044609,W,102.0,5.633167
22343,071203.90,43.771958,N,-0.044616,W,102.0,5.650143
22344,071204.00,43.771960,N,-0.044622,W,102.0,5.655288
...,...,...,...,...,...,...,...
42943,074720.40,43.771037,N,-0.040149,W,98.1,8.270209
42944,074720.50,43.771032,N,-0.040142,W,98.1,8.298503
42945,074720.60,43.771027,N,-0.040135,W,98.1,8.276382
42946,074720.70,43.771022,N,-0.040127,W,98.1,8.287700


In [6]:
#Plot om te checken of het is gelukt en ref met maps
#Het is gelukt.

%matplotlib qt

# Definieer een functie voor een 2D-plot
def plot_2d_height_profile(latitudes, longitudes, hoogtes):
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(longitudes, latitudes, c=hoogtes, cmap='viridis', s=10, alpha=0.8)
    plt.scatter(-0.039853, 43.770328, color='red', s=100, marker='o', label='Specifiek punt')
    plt.colorbar(scatter, label='Hoogte (m)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('2D Hoogteprofiel')
    plt.grid(True)
    plt.show()

# Voorbeeld dataframe (hier kun je je eigen gefilterde data inladen)
# Gebruik de functie met de gefilterde data
plot_2d_height_profile(filtered_df['latitude'], filtered_df['longitude'], filtered_df['hoogte'])


In [8]:
#Filter voor de rechte stukken.

# Set initial flag and empty list
gevonden = False
rechtestuk_rijen_hoogte = []

# Define the coordinates for the two rectangular areas
lat_min, lat_max = 7e-5 + 4.37724e1, 8.6e-5 + 4.37724e1
lon_min, lon_max = -0.04224, -0.04221

lat_min2, lat_max2 = 70e-5 + 4.377e1, 74e-5 + 4.377e1
lon_min2, lon_max2 = -0.03982, -0.03975


# Iterate over each row in the DataFrame (replace df with your actual DataFrame)
for i in range(len(filtered_df)):
    lat = filtered_df['latitude'].iloc[i]  # Latitude of the i-th row
    lon = filtered_df['longitude'].iloc[i]  # Longitude of the i-th row
   
    # Check if the point is within the first box
    if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
        if not gevonden:  # Append only the first time a point is found in the first box
            rechtestuk_rijen_hoogte.append(i)
            gevonden = True  # Set 'gevonden' flag to True

    # Check if the point is within the second box after finding a point in the first box
    if gevonden and lat_min2 <= lat <= lat_max2 and lon_min2 <= lon <= lon_max2:
        rechtestuk_rijen_hoogte.append(i)
        gevonden = False  # Reset 'gevonden' flag to False after processing the second box
        
    #Optionally, you can print out each step to debug
    # print(f"i: {i}, lat: {lat}, lon: {lon}, gevonden: {gevonden}, rechtestuk_rijen: {rechtestuk_rijen}")

# Convert the list to a numpy array if needed

# Print the results
print("Indices of points in the two boxes:", rechtestuk_rijen_hoogte)


Indices of points in the two boxes: [962, 1282, 3058, 3356, 5066, 5370, 7068, 7372, 9238, 9561, 11378, 11698, 13580, 13908, 15782, 16107, 18039, 18370, 20317]


In [9]:
#Check of het is gelukt met de rechte stukken
#Het is gelukt
%matplotlib qt

import matplotlib.pyplot as plt

def plot_hoogteprofiel_2d(latitudes, longitudes, hoogtes):
    # Increase figure size by setting the figsize
    fig, ax = plt.subplots(figsize=(12, 8))  # Adjust the width and height as needed
    scatter = ax.scatter(longitudes, latitudes, c=hoogtes, cmap='viridis', marker='o')
    ax.set_title('Hoogteprofiel van racebaan Nogaro', fontsize=16)
    ax.set_xlabel('Longitude', fontsize=14)
    ax.set_ylabel('Latitude', fontsize=14)
    
    # Create the colorbar
    cbar = fig.colorbar(scatter, ax=ax)
    cbar.set_label('Hoogte boven zeeniveau (m)', fontsize=12)

    # Plot the rectangular boundaries
    plt.plot(
        [lon_min, lon_max, lon_max, lon_min, lon_min],
        [lat_min, lat_min, lat_max, lat_max, lat_min],
        color='red', linestyle='--', linewidth=2, label="Boundary Square")

    plt.plot(
        [lon_min2, lon_max2, lon_max2, lon_min2, lon_min2],
        [lat_min2, lat_min2, lat_max2, lat_max2, lat_min2],
        color='red', linestyle='--', linewidth=2, label="Boundary Square")

    for i in range(len(latitudes)):
        lat = latitudes.iloc[i]
        lon = longitudes.iloc[i]
        #plt.text(lon, lat, str(i), fontsize=8, color='black')

    # Enable grid
    plt.grid(True)
    
    # Show the plot
    plt.show()

# Assuming `df` is your DataFrame with columns: 'latitude', 'longitude', and 'hoogte'
plot_hoogteprofiel_2d(filtered_df['latitude'], filtered_df['longitude'], filtered_df['hoogte'])


In [11]:
#Check afstand tussen begin en eind rechte stuk
import math

# Jouw haversine functie
def haversine(lat1, lon1, lat2, lon2, radius=6371000):  # Radius in meters
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in meters
    distance = radius * c
    return distance

# Omzetten van 'latitude' en 'longitude' naar correcte waarden (rekening houdend met N/S en E/W)
def correct_coordinates(lat, lat_dir, lon, lon_dir):
    if lat_dir == 'S':
        lat = -lat
    if lon_dir == 'W':
        lon = -lon
    return lat, lon

# Functie om de afstand te berekenen tussen twee rijen in de DataFrame
def calculate_distance_between_rows(df, row_index1, row_index2):
    # Haal de coördinaten en richting op voor de twee rijen
    lat1, lon1 = correct_coordinates(
        df.loc[row_index1, 'latitude'], df.loc[row_index1, 'lat_direction'],
        df.loc[row_index1, 'longitude'], df.loc[row_index1, 'lon_direction']
    )
    lat2, lon2 = correct_coordinates(
        df.loc[row_index2, 'latitude'], df.loc[row_index2, 'lat_direction'],
        df.loc[row_index2, 'longitude'], df.loc[row_index2, 'lon_direction']
    )
    
    # Bereken de afstand met de haversine functie
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance

# Voorbeeld van hoe de functie te gebruiken
# Stel dat filtered_df al je DataFrame is
row_index1 = 962+22340  # Vervang dit door de index van de eerste rij
row_index2 = 1282+22340  # Vervang dit door de index van de tweede rij

# Bereken de afstand tussen de twee punten
afstand = calculate_distance_between_rows(filtered_df, row_index1, row_index2)
print(f"De afstand tussen rij {row_index1} en rij {row_index2} is: {afstand} meter")


De afstand tussen rij 23302 en rij 23622 is: 275.5040146481721 meter


In [12]:
#Bereken delta x en delta h overal en laat deze in een aparte kolom sommeren
# Jouw haversine functie
def haversine(lat1, lon1, lat2, lon2, radius=6371000):  # Radius in meters
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in meters
    distance = radius * c
    return distance

# Omzetten van 'latitude' en 'longitude' naar correcte waarden (rekening houdend met N/S en E/W)
def correct_coordinates(lat, lat_dir, lon, lon_dir):
    if lat_dir == 'S':
        lat = -lat
    if lon_dir == 'W':
        lon = -lon
    return lat, lon

# Functie om de afstand te berekenen tussen twee rijen in de DataFrame
def calculate_distance_between_rows(df, row_index1, row_index2):
    # Haal de coördinaten en richting op voor de twee rijen
    lat1, lon1 = correct_coordinates(
        df.loc[row_index1, 'latitude'], df.loc[row_index1, 'lat_direction'],
        df.loc[row_index1, 'longitude'], df.loc[row_index1, 'lon_direction']
    )
    lat2, lon2 = correct_coordinates(
        df.loc[row_index2, 'latitude'], df.loc[row_index2, 'lat_direction'],
        df.loc[row_index2, 'longitude'], df.loc[row_index2, 'lon_direction']
    )
    
    # Bereken de afstand met de haversine functie
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance

# Functie om hoogteverschil te berekenen tussen twee rijen
def calculate_height_difference(df, row_index1, row_index2):
    hoogte1 = df.loc[row_index1, 'hoogte']
    hoogte2 = df.loc[row_index2, 'hoogte']
    return hoogte2 - hoogte1

# Bereken de afstand en hoogteverschil voor elke rij van 23302 tot en met 23622
rows = range(23302, 23623)
afstanden = []
hoogteverschillen = []

for i in range(len(rows) - 1):
    row_index1 = rows[i]
    row_index2 = rows[i + 1]
    
    # Bereken afstand en hoogteverschil
    afstand = calculate_distance_between_rows(filtered_df, row_index1, row_index2)
    hoogteverschil = calculate_height_difference(filtered_df, row_index1, row_index2)
    
    afstanden.append(afstand)
    hoogteverschillen.append(hoogteverschil)

# Maak een DataFrame met de resultaten
hoogte_vs_afstand = pd.DataFrame({
    'afstand': afstanden,
    'hoogteverschil': hoogteverschillen
})

# Voeg cumulatieve afstand en cumulatief hoogteverschil toe
hoogte_vs_afstand['cumulatieve_afstand'] = hoogte_vs_afstand['afstand'].cumsum()
hoogte_vs_afstand['cumulatief_hoogteverschil'] = hoogte_vs_afstand['hoogteverschil'].cumsum()

# Print de DataFrame
hoogte_vs_afstand


,afstand,hoogteverschil,cumulatieve_afstand,cumulatief_hoogteverschil
0,0.858225,0.0,0.858225,0.0
1,0.869951,0.0,1.728176,0.0
2,0.858225,0.0,2.586401,0.0
3,0.869951,0.0,3.456352,0.0
4,0.847833,0.0,4.304185,0.0
...,...,...,...,...
315,0.800074,0.0,272.624506,-1.2
316,0.800074,0.0,273.424580,-1.2
317,0.792314,0.0,274.216894,-1.2
318,0.792314,0.0,275.009208,-1.2


In [13]:
#hoogte vs afstand met fitvergelijking
hoogte_vs_afstand = pd.DataFrame(hoogte_vs_afstand)

# Data voor de plot
x = hoogte_vs_afstand['cumulatieve_afstand']
y = hoogte_vs_afstand['cumulatief_hoogteverschil']

# Zevende graads polynoomfit
degree = 7
coefficients_7 = np.polyfit(x, y, degree)
polynomial_7 = np.poly1d(coefficients_7)
y_fit_7 = polynomial_7(x)

# Plot de scatter en de fit
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Data punten')
plt.plot(x, y_fit_7, color='purple', linestyle='-', linewidth=2, label=f'Beste fit (graad {degree})')

# Formule van de polynoomfit genereren
fit_equation_7 = "y = " + " + ".join(
    [f"{coef:.2e}x^{degree - i}" if degree - i > 0 else f"{coef:.2e}" for i, coef in enumerate(coefficients_7)]
)

# Formule toevoegen aan de plot
plt.text(
    0.05 * max(x),
    0.85 * max(y),
    fit_equation_7,
    color='purple',
    fontsize=10,
    bbox=dict(facecolor='white', alpha=0.6)
)

# Labels en titel
plt.xlabel('Cumulatieve Afstand (m)', fontsize=14)
plt.ylabel('Cumulatief Hoogteverschil (m)', fontsize=14)
plt.title('Cumulatief Hoogteverschil vs. Cumulatieve Afstand', fontsize=16)
plt.grid(visible=True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12, loc='lower left')
plt.tight_layout()
plt.show()

# Print de formule
print(fit_equation_7)


y = -5.11e-16x^7 + 5.10e-13x^6 + -2.04e-10x^5 + 4.16e-08x^4 + -4.61e-06x^3 + 2.57e-04x^2 + -9.02e-03x^1 + -2.21e-03


In [14]:
#formule voor h, pas x_fit aan om het aantal datapunten te veranderen
x_fit = np.linspace(0, 275, 320)
h = polynomial_7(x_fit)

# Plot the fit equation from x = 0 to x = 275
plt.figure(figsize=(10, 6))
plt.plot(x_fit, h, color='purple', linestyle='-', linewidth=2, label=f'Polynoom fit (graad {degree})')
plt.grid(visible=True, linestyle='--', alpha=0.7)
#Add fit equation to the plot
plt.text(
    0.05 * max(x_fit),
    0.85 * max(y_fit_7),
    fit_equation_7,
    color='purple',
    fontsize=10,
    bbox=dict(facecolor='white', alpha=0.7)
)
plt.xlabel('Cumulatieve Afstand (m)', fontsize=14)
plt.ylabel('Cumulatief Hoogteverschil (m)', fontsize=14)
plt.title('Fit voor hoogte profiel rechte stuk', fontsize=16)

plt.show()

In [15]:
#Inladen motordriver

directory = "C:/Users\qvled\OneDrive - HvA\Logfiles\H2A/2023/Nogaro/20230525_h2a_poging1/0002/"
file_motordriver ="5.csv"
data_file_motordriver = lib.DataInladenMotorDriver(f'{directory}{file_motordriver}')

#Fix de longitude
data_file_motordriver["GPS longitude, in graden"] = data_file_motordriver["GPS longitude, in graden"]* -1

#Voeg wielsnelheid in m/s toe
data_file_motordriver.loc[:, "Wielsnelheid, in m/s"] = data_file_motordriver["Wielsnelheid, in km/h"] / 3.6

In [16]:
#Filter motordriver data van race begin tot race einde

def filter_track_data_with_start_and_end(data_file_motordriver, start_row, end_row):
    """
    Filter de data handmatig vanaf een opgegeven startrij tot een opgegeven eindrij.
    
    Parameters:
        data_file_motordriver (DataFrame): Het originele DataFrame.
        start_row (int): Het indexnummer van de rij waarmee het filteren moet beginnen.
        end_row (int): Het indexnummer van de rij waarmee het filteren moet eindigen.
        
    Returns:
        DataFrame: Het gefilterde DataFrame vanaf start_row tot end_row.
    """
    # Zorg ervoor dat de start_row en end_row binnen de geldige indexrange vallen
    if start_row < 0 or start_row >= len(data_file_motordriver):
        raise ValueError("De opgegeven start-row valt buiten de geldige indexrange.")
    if end_row < 0 or end_row >= len(data_file_motordriver):
        raise ValueError("De opgegeven end-row valt buiten de geldige indexrange.")
    
    # Controleer dat start_row kleiner of gelijk is aan end_row
    if start_row > end_row:
        raise ValueError("De start-row moet kleiner zijn dan of gelijk zijn aan de eind-row.")
    
    # Filter het DataFrame tussen start_row en end_row
    return data_file_motordriver.iloc[start_row:end_row + 1]

# Handmatige specificatie van start- en eindrij
start_row = 223257  # Begin vanaf rij 223257
end_row = 479444    # Eindig bij rij 250000

# Filter de data
filtered_motordriver = filter_track_data_with_start_and_end(data_file_motordriver, start_row, end_row)

# Print het gefilterde DataFrame
filtered_motordriver


,Datalogger port,"Dataloggertijd, in s",Format header (>03|04),Tijd sinds laatste herstart motordriver (s),Spanning over de motor (V),Stroom door de motor (A),Vermogen geleverd aan de motor (W),Energie geleverd aan de motor sinds reset (J),Spanning aan de ingang van motordriver (V),Stroom door de ingang van motordriver (A),...,Doelsnelheid van cruise control,"Toestand cruise control2-knop (aan=1, uit=0)",Tijdstip laatste verandering cruise control2-knop,"GPS longitude, in graden","GPS latitude, in graden",GPS direction,"GPS speed, in km/h","GPS time, in seconden sinds het begin van de huidige GPS-week",Checksum,"Wielsnelheid, in m/s"
223257,5,4540.1490,03|04,898.7554,2.689,9.883,36.8,-1340,31.898,1.225,...,0.000,0,0.000,-2.4802,4346.2552,333.0,1.7,73857.4,NaN,0.355278
223258,5,4540.1571,03|04,898.7634,3.127,10.037,43.2,-1340,31.894,1.325,...,0.000,0,0.000,-2.4802,4346.2553,333.0,1.7,73857.4,NaN,0.354167
223259,5,4540.1652,03|04,898.7716,3.702,10.203,51.8,-1339,31.889,1.454,...,0.000,0,0.000,-2.4802,4346.2553,339.2,1.7,73857.6,NaN,0.371389
223260,5,4540.1734,03|04,898.7796,3.541,10.329,50.1,-1339,31.889,1.499,...,0.000,0,0.000,-2.4802,4346.2553,339.2,1.7,73857.6,NaN,0.389444
223261,5,4540.1815,03|04,898.7878,3.242,10.222,46.2,-1339,31.894,1.391,...,0.000,0,0.000,-2.4802,4346.2553,339.2,1.7,73857.6,NaN,0.406944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479440,5,6774.8277,03|04,3133.0471,0.021,-0.085,-0.0,181490,30.445,-0.039,...,20.833,0,3073.986,-2.4679,4346.2367,323.4,3.0,81612.0,NaN,1.195278
479441,5,6774.8364,03|04,3133.0559,0.020,-0.084,-0.0,181490,30.445,-0.042,...,20.833,0,3073.986,-2.4679,4346.2367,323.4,3.0,81612.0,NaN,1.176111
479442,5,6774.8451,03|04,3133.0647,0.020,-0.084,-0.0,181490,30.445,-0.045,...,20.833,0,3073.986,-2.4679,4346.2367,323.4,3.0,81612.0,NaN,1.141389
479443,5,6774.8538,03|04,3133.0735,0.020,-0.083,-0.0,181490,30.445,-0.048,...,20.833,0,3073.986,-2.4679,4346.2367,323.4,3.0,81612.0,NaN,1.112500


In [17]:
# Functie om graden minuten naar decimale graden om te rekenen
def convert_to_decimal_degrees(value):
    if pd.isna(value):
        return None
    degrees = int(value // 100)  # De gehele gradenwaarde
    minutes = value % 100  # De minutenwaarde
    decimal_degrees = degrees + (minutes / 60)
    return decimal_degrees

# Pas de functie toe op latitude en longitude kolommen, waarbij NaN-waarden worden behouden
filtered_motordriver['GPS latitude, in graden'] = filtered_motordriver['GPS latitude, in graden'].apply(lambda x: convert_to_decimal_degrees(x) if pd.notna(x) else None)
filtered_motordriver['GPS longitude, in graden'] = filtered_motordriver['GPS longitude, in graden'].apply(lambda x: -convert_to_decimal_degrees(abs(x)) if pd.notna(x) and x < 0 else convert_to_decimal_degrees(x) if pd.notna(x) else None)

filtered_motordriver

C:\Users\qvled\AppData\Local\Temp\ipykernel_10264\2689420534.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_motordriver['GPS latitude, in graden'] = filtered_motordriver['GPS latitude, in graden'].apply(lambda x: convert_to_decimal_degrees(x) if pd.notna(x) else None)
C:\Users\qvled\AppData\Local\Temp\ipykernel_10264\2689420534.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_motordriver['GPS longitude, in graden'] = filtered_motordriver['GPS longitude, in graden'].apply(lambda x

,Datalogger port,"Dataloggertijd, in s",Format header (>03|04),Tijd sinds laatste herstart motordriver (s),Spanning over de motor (V),Stroom door de motor (A),Vermogen geleverd aan de motor (W),Energie geleverd aan de motor sinds reset (J),Spanning aan de ingang van motordriver (V),Stroom door de ingang van motordriver (A),...,Doelsnelheid van cruise control,"Toestand cruise control2-knop (aan=1, uit=0)",Tijdstip laatste verandering cruise control2-knop,"GPS longitude, in graden","GPS latitude, in graden",GPS direction,"GPS speed, in km/h","GPS time, in seconden sinds het begin van de huidige GPS-week",Checksum,"Wielsnelheid, in m/s"
223257,5,4540.1490,03|04,898.7554,2.689,9.883,36.8,-1340,31.898,1.225,...,0.000,0,0.000,-0.041337,43.770920,333.0,1.7,73857.4,NaN,0.355278
223258,5,4540.1571,03|04,898.7634,3.127,10.037,43.2,-1340,31.894,1.325,...,0.000,0,0.000,-0.041337,43.770922,333.0,1.7,73857.4,NaN,0.354167
223259,5,4540.1652,03|04,898.7716,3.702,10.203,51.8,-1339,31.889,1.454,...,0.000,0,0.000,-0.041337,43.770922,339.2,1.7,73857.6,NaN,0.371389
223260,5,4540.1734,03|04,898.7796,3.541,10.329,50.1,-1339,31.889,1.499,...,0.000,0,0.000,-0.041337,43.770922,339.2,1.7,73857.6,NaN,0.389444
223261,5,4540.1815,03|04,898.7878,3.242,10.222,46.2,-1339,31.894,1.391,...,0.000,0,0.000,-0.041337,43.770922,339.2,1.7,73857.6,NaN,0.406944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479440,5,6774.8277,03|04,3133.0471,0.021,-0.085,-0.0,181490,30.445,-0.039,...,20.833,0,3073.986,-0.041132,43.770612,323.4,3.0,81612.0,NaN,1.195278
479441,5,6774.8364,03|04,3133.0559,0.020,-0.084,-0.0,181490,30.445,-0.042,...,20.833,0,3073.986,-0.041132,43.770612,323.4,3.0,81612.0,NaN,1.176111
479442,5,6774.8451,03|04,3133.0647,0.020,-0.084,-0.0,181490,30.445,-0.045,...,20.833,0,3073.986,-0.041132,43.770612,323.4,3.0,81612.0,NaN,1.141389
479443,5,6774.8538,03|04,3133.0735,0.020,-0.083,-0.0,181490,30.445,-0.048,...,20.833,0,3073.986,-0.041132,43.770612,323.4,3.0,81612.0,NaN,1.112500


In [18]:
plt.title('Motordriver GPS tegen Hoogteprofiel GPS')
plt.ylabel('Longitude')
plt.xlabel('Latitude')
plt.scatter(filtered_motordriver['GPS longitude, in graden'], filtered_motordriver['GPS latitude, in graden'], color ='r', label= 'Filtered_motordriver')
plt.scatter(filtered_df['longitude'], filtered_df['latitude'], color='b', label='Filtered DF')
plt.legend()
plt.grid(visible=True, linestyle='--', alpha=0.7)
plt.show()

In [20]:
import pandas as pd

# Voorbeeld dataframe 'finish' met gelijke lengtes voor alle kolommen
data = {
    'GPS latitude, in graden': [0.226 + 4.346e3, 0.237 + 4.346e3],
    'GPS longitude, in graden': [-2.4250, -2.4225],  # Longitude bevatte al negatieve waarden
    'Latitude richting': ['N', 'N'],
    'Longitude richting': ['W', 'W']
}

# Maak een DataFrame van de data
finish = pd.DataFrame(data)

# Functie om graden minuten naar decimale graden om te rekenen
def convert_to_decimal_degrees(value):
    if pd.isna(value):
        return None
    degrees = int(value // 100)  # De gehele gradenwaarde
    minutes = value % 100  # De minutenwaarde
    decimal_degrees = degrees + (minutes / 60)
    return decimal_degrees

# Pas de functie toe op latitude en longitude kolommen, waarbij NaN-waarden worden behouden
finish['GPS latitude, in graden'] = finish['GPS latitude, in graden'].apply(lambda x: convert_to_decimal_degrees(x) if pd.notna(x) else None)
finish['GPS longitude, in graden'] = finish['GPS longitude, in graden'].apply(lambda x: -convert_to_decimal_degrees(abs(x)) if pd.notna(x) and x < 0 else convert_to_decimal_degrees(x) if pd.notna(x) else None)

print(finish)


   GPS latitude, in graden  GPS longitude, in graden Latitude richting  \
0                43.770433                 -0.040417                 N   
1                43.770617                 -0.040375                 N   

  Longitude richting  
0                  W  
1                  W  


In [21]:
#Finish rijen

gevonden = False
finish_rijen = np.array([])

lat_min, lat_max = -0.040417, -0.040375
lon_min, lon_max = 43.770433, 43.770617 

# Kolomnamen
yas = 'GPS latitude, in graden'
xas = 'GPS longitude, in graden'
afstand_rondje = 1588.84925219  # Afstand van een rondje in meters

# Filter rows where latitude, longitude, and the distance are non-zero
filtered_motordriver = filtered_motordriver[filtered_motordriver[xas] != 0]
filtered_motordriver = filtered_motordriver[filtered_motordriver[yas] != 0]
filtered_motordriver = filtered_motordriver[filtered_motordriver['Afgelegde afstand sinds laatste herstart motordriver (m)'] != 0]

# Itereer over elke rij in de DataFrame
for i in range(len(filtered_motordriver)):
    lat = filtered_motordriver[xas].iloc[i]  # Latitude van de i-de rij
    lon = filtered_motordriver[yas].iloc[i]  # Longitude van de i-de rij
    afstand = filtered_motordriver["Afgelegde afstand sinds laatste herstart motordriver (m)"].iloc[i]

    # Controleer of het punt binnen het vierkant valt
    if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
        if not gevonden:  # Voeg alleen toe als dit de eerste keer is dat we het punt vinden
            finish_rijen = np.append(finish_rijen, i)
            gevonden = True  # Vlag op 'gevonden' zetten
            start_afstand = afstand  # Opslaan van de startafstand bij het eerste punt

    # Controleer of de afgelegde afstand groter is dan een volledige ronde
    if gevonden and afstand >= start_afstand + (afstand_rondje * (len(finish_rijen))):
        finish_rijen = np.append(finish_rijen, i)

print(finish_rijen)
afstanden = np.array([])  # Maak een lege array voor afstanden


[ 26251.  49486.  74393. 100200. 124732. 149852. 174488. 200208. 226012.
 255494.]


In [22]:
#Check of het is gelukt met de rondes
#Het is gelukt
%matplotlib qt

def plot_hoogteprofiel_2d(latitudes, longitudes, hoogtes):
    # Increase figure size by setting the figsize
    fig, ax = plt.subplots(figsize=(12, 8))  # Adjust the width and height as needed
    scatter = ax.scatter(longitudes, latitudes, c=hoogtes, cmap='viridis', marker='o')
    ax.set_title('Hoogteprofiel van racebaan Nogaro', fontsize=16)
    ax.set_xlabel('Longitude', fontsize=14)
    ax.set_ylabel('Latitude', fontsize=14)
    
    # Create the colorbar
    cbar = fig.colorbar(scatter, ax=ax)
    cbar.set_label('Hoogte boven zeeniveau (m)', fontsize=12)

    # Plot the rectangular boundaries
    plt.plot(
        [lon_min, lon_max, lon_max, lon_min, lon_min],
        [lat_min, lat_min, lat_max, lat_max, lat_min],
        color='red', linestyle='--', linewidth=2, label="Boundary Square")

    for i in range(len(latitudes)):
        lat = latitudes.iloc[i]
        lon = longitudes.iloc[i]
        #plt.text(lon, lat, str(i), fontsize=8, color='black')

    # Enable grid
    plt.grid(True)
    
    # Show the plot
    plt.show()

# Assuming `df` is your DataFrame with columns: 'latitude', 'longitude', and 'hoogte'
plot_hoogteprofiel_2d(filtered_df['latitude'], filtered_df['longitude'], filtered_df['hoogte'])


In [23]:
#Maken df's per ronde motordriver
begin_eind_paren2 = [
    (0, 26251), 
    (26251, 49486), 
    (49486, 74393), 
    (74393, 100200),
    (100200, 124732), 
    (124732, 149852), 
    (149852, 174488), 
    (174488, 200208),
    (200208, 226012), 
    (226012, 255494)
]

# Maak een lege lijst om de rondes-dataframes op te slaan
rondes = []

# Loop door elke (begin, eind) paar en maak een DataFrame subset
for i, (begin, eind) in enumerate(begin_eind_paren2):
    df_segment = filtered_motordriver.iloc[begin:eind]
    rondes.append(df_segment)

# Je kunt nu elke ronde benaderen als rondes[0], rondes[1], etc.
df_ronde1 = rondes[0]
df_ronde2 = rondes[1]
df_ronde3 = rondes[2]
df_ronde4 = rondes[3]
df_ronde5 = rondes[4]
df_ronde6 = rondes[5]
df_ronde7 = rondes[6]
df_ronde8 = rondes[7]
df_ronde9 = rondes[8]
df_ronde10 = rondes[9]

# Print het resultaat voor de eerste ronde om te controleren
df_ronde6


,Datalogger port,"Dataloggertijd, in s",Format header (>03|04),Tijd sinds laatste herstart motordriver (s),Spanning over de motor (V),Stroom door de motor (A),Vermogen geleverd aan de motor (W),Energie geleverd aan de motor sinds reset (J),Spanning aan de ingang van motordriver (V),Stroom door de ingang van motordriver (A),...,Doelsnelheid van cruise control,"Toestand cruise control2-knop (aan=1, uit=0)",Tijdstip laatste verandering cruise control2-knop,"GPS longitude, in graden","GPS latitude, in graden",GPS direction,"GPS speed, in km/h","GPS time, in seconden sinds het begin van de huidige GPS-week",Checksum,"Wielsnelheid, in m/s"
347989,5,5617.1410,03|04,1975.5612,20.513,8.113,178.6,96685,27.494,6.606,...,21.953,0,1840.759,-0.040665,43.770847,325.9,25.7,75654.4,NaN,7.213056
347990,5,5617.1498,03|04,1975.5699,20.556,8.133,179.8,96686,27.493,6.603,...,21.953,0,1840.759,-0.040665,43.770847,325.9,25.7,75654.4,NaN,7.131667
347991,5,5617.1586,03|04,1975.5789,20.435,8.116,179.4,96688,27.492,6.613,...,21.953,0,1840.759,-0.040665,43.770847,325.9,25.7,75654.4,NaN,7.202222
347992,5,5617.1674,03|04,1975.5876,20.648,8.120,180.6,96689,27.491,6.611,...,21.953,0,1840.759,-0.040665,43.770847,325.9,25.7,75654.4,NaN,7.140556
347993,5,5617.1761,03|04,1975.5964,20.702,8.080,180.2,96691,27.489,6.624,...,21.953,0,1840.759,-0.040665,43.770847,325.9,25.7,75654.4,NaN,7.187778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373104,5,5837.5367,03|04,2195.9187,21.498,8.001,182.8,114180,27.361,6.734,...,29.107,0,2131.384,-0.040823,43.770935,322.7,26.1,80034.8,NaN,7.497500
373105,5,5837.5455,03|04,2195.9275,21.908,8.061,187.0,114182,27.360,6.722,...,29.107,0,2131.384,-0.040823,43.770935,322.7,26.1,80034.8,NaN,7.290278
373106,5,5837.5543,03|04,2195.9363,21.476,8.042,184.1,114183,27.359,6.731,...,29.107,0,2131.384,-0.040823,43.770935,322.7,26.1,80034.8,NaN,7.505556
373107,5,5837.5631,03|04,2195.9453,21.311,8.051,182.8,114185,27.358,6.742,...,29.107,0,2131.384,-0.040823,43.770935,322.7,26.1,80034.8,NaN,7.273611


In [24]:
#Filter voor recht stuk in ronde 6

# Set initial flag and empty list
gevonden = False
rechtestuk_rijen_hoogte = []

# Define the coordinates for the two rectangular areas
lat_min, lat_max = 7e-5 + 4.37724e1, 8.6e-5 + 4.37724e1
lon_min, lon_max = -0.04224, -0.04221

lat_min2, lat_max2 = 70e-5 + 4.377e1, 74e-5 + 4.377e1
lon_min2, lon_max2 = -0.03982, -0.03975


# Iterate over each row in the DataFrame (replace df with your actual DataFrame)
for i in range(len(df_ronde6)):
    lat = df_ronde6['GPS latitude, in graden'].iloc[i]  # Latitude of the i-th row
    lon = df_ronde6['GPS longitude, in graden'].iloc[i]  # Longitude of the i-th row
   
    # Check if the point is within the first box
    if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
        if not gevonden:  # Append only the first time a point is found in the first box
            rechtestuk_rijen_hoogte.append(i)
            gevonden = True  # Set 'gevonden' flag to True

    # Check if the point is within the second box after finding a point in the first box
    if gevonden and lat_min2 <= lat <= lat_max2 and lon_min2 <= lon <= lon_max2:
        rechtestuk_rijen_hoogte.append(i)
        gevonden = False  # Reset 'gevonden' flag to False after processing the second box
        
    #Optionally, you can print out each step to debug
    # print(f"i: {i}, lat: {lat}, lon: {lon}, gevonden: {gevonden}, rechtestuk_rijen: {rechtestuk_rijen}")

# Convert the list to a numpy array if needed

# Print the results
print("Indices of points in the two boxes:", rechtestuk_rijen_hoogte)



Indices of points in the two boxes: [18594, 22484]


In [25]:
#Bepalen hoeveel waardes van h we nodig hebben
getallen_tussen_rechtestuk_echt = []

# Calculate the number of points in each straight section
for i in range(0, len(rechtestuk_rijen_hoogte) - 1, 2):
    lengtes_rechte_stuk_echt = rechtestuk_rijen_hoogte[i + 1] - rechtestuk_rijen_hoogte[i]
    getallen_tussen_rechtestuk_echt.append(lengtes_rechte_stuk_echt)

print(getallen_tussen_rechtestuk_echt)

[3890]


In [26]:
# Dictionary to store datasets for each segment
#formule voor h, pas x_fit aan om het aantal datapunten te veranderen
x_fit = np.linspace(0, 275, 3891)
h = polynomial_7(x_fit)

# Plot the fit equation from x = 0 to x = 275
plt.figure(figsize=(10, 6))
plt.plot(x_fit, h, color='purple', linestyle='-', linewidth=2, label=f'Polynoom fit (graad {degree})')
plt.grid(visible=True, linestyle='--', alpha=0.7)
#Add fit equation to the plot
plt.text(
    0.05 * max(x_fit),
    0.85 * max(y_fit_7),
    fit_equation_7,
    color='purple',
    fontsize=10,
    bbox=dict(facecolor='white', alpha=0.7)
)
plt.xlabel('Cumulatieve Afstand (m)', fontsize=14)
plt.ylabel('Cumulatief Hoogteverschil (m)', fontsize=14)
plt.title('Fit voor hoogte profiel rechte stuk (3890 datapunten)', fontsize=16)

plt.show() 

df_h = pd.DataFrame({'hoogteverschil (m)': h})
# Voeg een kolom toe met het verschil in hoogte (delta h)
df_h['delta h (m)'] = df_h['hoogteverschil (m)'].diff().fillna(0)


In [27]:
print(df_ronde6.loc[18594+347989])

Datalogger port                                                          5
Dataloggertijd, in s                                             5780.3688
Format header (>03|04)                                               03|04
Tijd sinds laatste herstart motordriver (s)                      2138.7607
Spanning over de motor (V)                                           0.024
Stroom door de motor (A)                                            -0.051
Vermogen geleverd aan de motor (W)                                    -0.0
Energie geleverd aan de motor sinds reset (J)                       111487
Spanning aan de ingang van motordriver (V)                          27.719
Stroom door de ingang van motordriver (A)                           -0.007
Vermogen geleverd aan de hele motordriver (W)                         -0.2
Energie geleverd aan de hele motordriver sinds reset (J)            120100
Motordriver temperatuur (C)                                         119.65
Motordriver-bord temperat

In [28]:
print(df_ronde6.loc[18594+347989+3800])

Datalogger port                                                          5
Dataloggertijd, in s                                             5813.5914
Format header (>03|04)                                               03|04
Tijd sinds laatste herstart motordriver (s)                      2171.9775
Spanning over de motor (V)                                           0.022
Stroom door de motor (A)                                            -0.066
Vermogen geleverd aan de motor (W)                                    -0.0
Energie geleverd aan de motor sinds reset (J)                       111487
Spanning aan de ingang van motordriver (V)                           29.66
Stroom door de ingang van motordriver (A)                            -0.02
Vermogen geleverd aan de hele motordriver (W)                         -0.6
Energie geleverd aan de hele motordriver sinds reset (J)            120080
Motordriver temperatuur (C)                                         119.44
Motordriver-bord temperat

In [29]:
df_h*-1

,hoogteverschil (m),delta h (m)
0,0.002207,-0.000000
1,0.002843,0.000636
2,0.003477,0.000634
3,0.004108,0.000631
4,0.004736,0.000628
...,...,...
3886,1.219704,0.000411
3887,1.220116,0.000412
3888,1.220529,0.000413
3889,1.220943,0.000414


In [30]:
import pandas as pd

# Berekenen zwaartekracht
massa = 90  # Mass in kg
g = 9.81  # Gravitational acceleration in m/s²

# Initial speed in m/s
begin_snelheid = df_ronde6.loc[18594 + 347989, "Wielsnelheid, in m/s"]
print(f"Initial speed: {begin_snelheid} m/s")

# Bereken de initiële kinetische energie
initiele_kinetische_energie = 0.5 * massa * begin_snelheid**2
print(f"Initiële kinetische energie: {initiele_kinetische_energie} J")

# Maak een nieuwe lege DataFrame df_e met dezelfde index als df_h
df_e = pd.DataFrame(index=df_h.index)

# Bereken potentiële energie per punt op basis van de hoogteverschillen (neem absolute waarde van delta h)
df_e['Potentiële Energie (J)'] = massa * g * df_h['delta h (m)'].abs()

# Bereken de omzetting van potentiële energie naar kinetische energie
df_e['Omzetting naar Kinetische Energie (J)'] = df_e['Potentiële Energie (J)']

# Bereken de cumulatieve som van de omzetting naar kinetische energie
df_e['Kinetische Energie Totaal (J)'] = df_e['Omzetting naar Kinetische Energie (J)'].cumsum()

# Voeg de initiële kinetische energie toe aan de hele kolom om de juiste startwaarde mee te nemen
df_e['Kinetische Energie Totaal (J)'] += initiele_kinetische_energie

# Print het resultaat voor controle
print(df_e.head())


Initial speed: 8.749444444444444 m/s
Initiële kinetische energie: 3444.8750138888886 J
   Potentiële Energie (J)  Omzetting naar Kinetische Energie (J)  \
0                0.000000                               0.000000   
1                0.561635                               0.561635   
2                0.559376                               0.559376   
3                0.557126                               0.557126   
4                0.554884                               0.554884   

   Kinetische Energie Totaal (J)  
0                    3444.875014  
1                    3445.436649  
2                    3445.996025  
3                    3446.553151  
4                    3447.108035  


In [31]:
start_index = 18594 + 347989
end_index = start_index + 3890

# Bereken de kinetische energie met de gemeten snelheden van df_ronde6
gemeten_snelheden = df_ronde6.loc[start_index:end_index, "Wielsnelheid, in m/s"]
df_e_kin = pd.DataFrame(index=gemeten_snelheden.index)  # Nieuwe DataFrame voor de gemeten kinetische energie
df_e_kin['Gemeten Snelheid (m/s)'] = gemeten_snelheden
df_e_kin['Kinetische Energie (J)'] = 0.5 * massa * df_e_kin['Gemeten Snelheid (m/s)']**2

df_e_kin

,Gemeten Snelheid (m/s),Kinetische Energie (J)
366583,8.749444,3444.875014
366584,8.542222,3283.630222
366585,8.686667,3395.618000
366586,8.606667,3333.362000
366587,8.586389,3317.673337
...,...,...
370469,7.874722,2790.506253
370470,7.788056,2729.421420
370471,7.904444,2811.610889
370472,7.721111,2682.700056


In [32]:
# Reset de index van df_ronde6 zodat deze begint vanaf 0
df_ronde6_reset = df_ronde6.reset_index()

# Bereken de nieuwe start en eind indexen op basis van de numerieke positie
start_pos = df_ronde6_reset[df_ronde6_reset['index'] == start_index].index[0]  # Vind de nieuwe positie na reset
end_pos = start_pos + 3890

# Selecteer de waarden voor de nieuwe kolom
df_e['Energie geleverd aan de motor (J)'] = df_ronde6_reset.iloc[start_pos:end_pos + 1]['Energie geleverd aan de hele motordriver sinds reset (J)'].values


In [33]:


# Bereken de delta van de geleverde energie tussen opeenvolgende rijen
df_e['Delta Energie geleverd aan de motor (J)'] = df_e['Energie geleverd aan de motor (J)'].diff().fillna(0)

# Bereken de gesommeerde waarde van de delta energie over alle rijen
df_e['Gesommeerde Delta Energie (J)'] = df_e['Delta Energie geleverd aan de motor (J)'].cumsum()

# Print de eerste paar rijen van df_e om de nieuwe kolommen te bekijken
df_e


,Potentiële Energie (J),Omzetting naar Kinetische Energie (J),Kinetische Energie Totaal (J),Energie geleverd aan de motor (J),Delta Energie geleverd aan de motor (J),Gesommeerde Delta Energie (J)
0,0.000000,0.000000,3444.875014,120100,0.0,0.0
1,0.561635,0.561635,3445.436649,120100,0.0,0.0
2,0.559376,0.559376,3445.996025,120100,0.0,0.0
3,0.557126,0.557126,3446.553151,120100,0.0,0.0
4,0.554884,0.554884,3447.108035,120100,0.0,0.0
...,...,...,...,...,...,...
3886,0.362610,0.362610,4519.802956,120079,0.0,-21.0
3887,0.363568,0.363568,4520.166524,120079,0.0,-21.0
3888,0.364532,0.364532,4520.531056,120079,0.0,-21.0
3889,0.365501,0.365501,4520.896557,120079,0.0,-21.0


In [37]:
# Plot de kinetische energie tegen de afgelegde afstand
x= np.linspace(0,275.5,3891)
plt.figure(figsize=(10, 6))
plt.plot(x, df_e['Kinetische Energie Totaal (J)'], label='Kinetische Energie in situatie zonder weerstand (berekend)')
plt.scatter(x, df_e_kin['Kinetische Energie (J)'], label='Kinetische Energie (gemeten)', color = 'r')



# Labels en titel toevoegen
plt.xlim(0,276)
plt.xticks(np.arange(0, 276, 25))  # Stel de x-as in stappen van 25
plt.xlabel('Afgelegde Afstand (m)')
plt.ylabel('Kinetische Energie (J)')
plt.title('Kinetische Energie over Afgelegde Afstand met delta h van 1.2 m (uitrollend)')
plt.legend()
plt.grid(True)

# Toon de grafiek
plt.show()

In [38]:
x = np.linspace(0, 275.5, 3891)
plt.figure(figsize=(10, 6))

plt.scatter(x, df_e['Gesommeerde Delta Energie (J)'], label='Energie geleverd aan motordriver (Gemeten)', color='g')
# Labels en titel toevoegen
plt.xlim(0, 276)
plt.xticks(np.arange(0, 276, 25))  # Stel de x-as in stappen van 25
plt.xlabel('Afgelegde Afstand (m)')
plt.ylabel('Energie (J)')
plt.title('Energie geleverd aan motordriver (meetdata) over Afgelegde Afstand met delta h van 1.2 m (uitrollend)')
plt.legend()
plt.grid(True)

# Toon de grafiek
plt.show()

In [39]:
#Plot geleverde energie aan motordriver erbij
# Maak de x-as voor de afgelegde afstand
x = np.linspace(0, 275.5, 3891)

# Plot de kinetische energie en de gesommeerde delta-energie tegen de afgelegde afstand
plt.figure(figsize=(10, 6))
plt.plot(x, df_e['Kinetische Energie Totaal (J)'], label='Kinetische Energie in situatie zonder weerstand (berekend)')
plt.scatter(x, df_e_kin['Kinetische Energie (J)'], label='Kinetische Energie (Berekend uit gemeten snelheid)', color='r')
plt.scatter(x, df_e['Gesommeerde Delta Energie (J)'], label='Energie geleverd aan motordriver (Gemeten)', color='g')

# Labels en titel toevoegen
plt.xlim(0, 276)
plt.xticks(np.arange(0, 276, 25))  # Stel de x-as in stappen van 25
plt.xlabel('Afgelegde Afstand (m)')
plt.ylabel('Energie (J)')
plt.title('Kinetische Energie en Gesommeerde Delta Energie over Afgelegde Afstand met delta h van 1.2 m (uitrollend)')
plt.legend()
plt.grid(True)

# Toon de grafiek
plt.show()

In [40]:
# Stel massa, zwaartekracht, en rolweerstandscoëfficiënt in
massa = 90  # Massa in kg
g = 9.81  # Zwaartekrachtversnelling in m/s²
C_r = 0.006  # Rolweerstandscoëfficiënt

# Definieer de x-as als linspace (afgelegde afstand in meters)
x = np.linspace(0, 275.5, 3891)  # 3891 stappen over een afstand van 275.5 meter

# Stel de horizontale afstand (\Delta x) per stap in (gebruik linspace met gelijke afstanden)
delta_x = x[1] - x[0]  # Afstand tussen opeenvolgende meetpunten

# Bereken de hellingshoek per stap op basis van hoogteverschil en horizontale afstand
df_e['Hoek (rad)'] = np.arctan(df_h['delta h (m)'] / delta_x)

# Bereken de normaalkracht per rij rekening houdend met de hellingshoek
df_e['Normaalkracht (N)'] = massa * g * np.cos(df_e['Hoek (rad)'])

# Bereken de rolweerstandskracht per rij met de formule: R_rol = F_n * C_r
df_e['Rolweerstand (N)'] = df_e['Normaalkracht (N)'] * C_r

# Bereken de energieverlies door rolweerstand per stap (E_rol = F_rol * delta_x)
df_e['Energieverlies door rolweerstand (J)'] = df_e['Rolweerstand (N)'] * delta_x

# Bereken de cumulatieve energieverlies door rolweerstand
df_e['Cumulatieve Energieverlies door rolweerstand (J)'] = df_e['Energieverlies door rolweerstand (J)'].cumsum()

df_e

,Potentiële Energie (J),Omzetting naar Kinetische Energie (J),Kinetische Energie Totaal (J),Energie geleverd aan de motor (J),Delta Energie geleverd aan de motor (J),Gesommeerde Delta Energie (J),Hoek (rad),Normaalkracht (N),Rolweerstand (N),Energieverlies door rolweerstand (J),Cumulatieve Energieverlies door rolweerstand (J)
0,0.000000,0.000000,3444.875014,120100,0.0,0.0,0.000000,882.900000,5.297400,0.375176,0.375176
1,0.561635,0.561635,3445.436649,120100,0.0,0.0,-0.008982,882.864388,5.297186,0.375161,0.750336
2,0.559376,0.559376,3445.996025,120100,0.0,0.0,-0.008946,882.864674,5.297188,0.375161,1.125497
3,0.557126,0.557126,3446.553151,120100,0.0,0.0,-0.008910,882.864957,5.297190,0.375161,1.500658
4,0.554884,0.554884,3447.108035,120100,0.0,0.0,-0.008874,882.865239,5.297191,0.375161,1.875819
...,...,...,...,...,...,...,...,...,...,...,...
3886,0.362610,0.362610,4519.802956,120079,0.0,-21.0,-0.005799,882.885155,5.297311,0.375169,1458.292981
3887,0.363568,0.363568,4520.166524,120079,0.0,-21.0,-0.005814,882.885076,5.297310,0.375169,1458.668151
3888,0.364532,0.364532,4520.531056,120079,0.0,-21.0,-0.005830,882.884997,5.297310,0.375169,1459.043320
3889,0.365501,0.365501,4520.896557,120079,0.0,-21.0,-0.005845,882.884917,5.297310,0.375169,1459.418490


In [41]:
# Plot de cumulatieve kinetische energie en het energieverlies door rolweerstand tegen de afgelegde afstand
plt.figure(figsize=(10, 6))

# Plot de kinetische energie zonder weerstand
plt.plot(x, df_e['Kinetische Energie Totaal (J)'], label='Kinetische Energie in situatie zonder weerstand (berekend)', color='b')

# Plot de cumulatieve energieverlies door rolweerstand
plt.plot(x, df_e['Cumulatieve Energieverlies door rolweerstand (J)'], label='Cumulatieve Energieverlies door rolweerstand', color='g')

# Labels en titel toevoegen
plt.xlabel('Afgelegde Afstand (m)')
plt.ylabel('Energie (J)')
plt.title('Kinetische Energie en Energieverlies door Rolweerstand over Afgelegde Afstand')
plt.legend()
plt.grid(True)

# Toon de grafiek
plt.show()


In [42]:
# Plot de cumulatieve kinetische energie en het energieverlies door rolweerstand tegen de afgelegde afstand
plt.figure(figsize=(10, 6))

# Plot de kinetische energie zonder weerstand
plt.plot(x, df_e['Kinetische Energie Totaal (J)'], label='Kinetische Energie theoretisch (berekend)', color='g')
plt.plot(x, df_e['Kinetische Energie Totaal (J)']-df_e['Cumulatieve Energieverlies door rolweerstand (J)'], label='Kinetische Energie in situatie met rolweerstand (berekend)', color='b')
plt.scatter(x, df_e_kin['Kinetische Energie (J)'], label='Kinetische Energie (Berekend uit gemeten snelheid)', color='r')

# Labels en titel toevoegen
plt.xlabel('Afgelegde Afstand (m)')
plt.ylabel('Energie (J)')
plt.title('Kinetische Energie en Energieverlies door Rolweerstand over Afgelegde Afstand')
plt.legend()
plt.grid(True)

# Toon de grafiek
plt.show()


In [46]:
# Print de lengte van 'gemeten_snelheden' en 'df_e' om te controleren of ze gelijk zijn
print(f"Lengte van gemeten_snelheden: {len(gemeten_snelheden)}, Lengte van df_e: {len(df_e)}")

# Controleer of er NaN-waarden zijn in 'gemeten_snelheden'
if isinstance(gemeten_snelheden, pd.Series):
    print(f"Aantal NaN-waarden in gemeten_snelheden: {gemeten_snelheden.isna().sum()}")

# Voeg de snelheden toe aan df_e, ongeacht de index
df_e['Snelheid (m/s)'] = gemeten_snelheden.values  # Gebruik de waarden zonder de index mee te nemen

# Print de eerste paar rijen van df_e om te controleren of de snelheden zijn toegevoegd
print(df_e['Snelheid (m/s)'].head())


Lengte van gemeten_snelheden: 3891, Lengte van df_e: 3891
Aantal NaN-waarden in gemeten_snelheden: 0
0    8.749444
1    8.542222
2    8.686667
3    8.606667
4    8.586389
Name: Snelheid (m/s), dtype: float64


In [48]:

# Stel de luchtweerstandscoëfficiënt in
C_d = 0.09  # Luchtweerstandscoëfficiënt

# Stel een constante luchtdichtheid in (rho, in kg/m^3), typisch 1.225 voor droge lucht op zeeniveau
rho = 1.225  # kg/m^3

# Stel de frontale oppervlakte van de auto in (in m²) als voorbeeldwaarde
A = 2.5  # Frontale oppervlakte in m² (typische waarde voor een auto)

# Werk opnieuw met de x-as als linspace voor de afgelegde afstand
x = np.linspace(0, 275.5, 3891)  # 3891 stappen over een afstand van 275.5 meter
df_e['Afstand (m)'] = x

# Bereken de luchtweerstandskracht per stap (F_lucht = 0.5 * C_d * rho * A * v^2) met de bijgewerkte afstand
df_e['Luchtweerstand (N)'] = 0.5 * C_d * rho * A * df_e['Snelheid (m/s)']**2

# Bereken het energieverlies door luchtweerstand per stap opnieuw (E_lucht = F_lucht * delta_x)
df_e['Energieverlies door luchtweerstand (J)'] = df_e['Luchtweerstand (N)'] * delta_x

# Bereken de cumulatieve energieverlies door luchtweerstand opnieuw
df_e['Cumulatieve Energieverlies door luchtweerstand (J)'] = df_e['Energieverlies door luchtweerstand (J)'].cumsum()

# Print de eerste paar rijen van df_e om de nieuwe kolommen te bekijken
df_e


,Potentiële Energie (J),Omzetting naar Kinetische Energie (J),Kinetische Energie Totaal (J),Energie geleverd aan de motor (J),Delta Energie geleverd aan de motor (J),Gesommeerde Delta Energie (J),Hoek (rad),Normaalkracht (N),Rolweerstand (N),Energieverlies door rolweerstand (J),Cumulatieve Energieverlies door rolweerstand (J),Snelheid (m/s),Afstand (m),Luchtweerstand (N),Energieverlies door luchtweerstand (J),Cumulatieve Energieverlies door luchtweerstand (J)
0,0.000000,0.000000,3444.875014,120100,0.0,0.0,0.000000,882.900000,5.297400,0.375176,0.375176,8.749444,0.000000,10.549930,0.747174,0.747174
1,0.561635,0.561635,3445.436649,120100,0.0,0.0,-0.008982,882.864388,5.297186,0.375161,0.750336,8.542222,0.070823,10.056118,0.712201,1.459374
2,0.559376,0.559376,3445.996025,120100,0.0,0.0,-0.008946,882.864674,5.297188,0.375161,1.125497,8.686667,0.141645,10.399080,0.736490,2.195864
3,0.557126,0.557126,3446.553151,120100,0.0,0.0,-0.008910,882.864957,5.297190,0.375161,1.500658,8.606667,0.212468,10.208421,0.722987,2.918852
4,0.554884,0.554884,3447.108035,120100,0.0,0.0,-0.008874,882.865239,5.297191,0.375161,1.875819,8.586389,0.283290,10.160375,0.719584,3.638436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,0.362610,0.362610,4519.802956,120079,0.0,-21.0,-0.005799,882.885155,5.297311,0.375169,1458.292981,7.874722,275.216710,8.545925,0.605245,2553.975806
3887,0.363568,0.363568,4520.166524,120079,0.0,-21.0,-0.005814,882.885076,5.297310,0.375169,1458.668151,7.788056,275.287532,8.358853,0.591996,2554.567802
3888,0.364532,0.364532,4520.531056,120079,0.0,-21.0,-0.005830,882.884997,5.297310,0.375169,1459.043320,7.904444,275.358355,8.610558,0.609822,2555.177624
3889,0.365501,0.365501,4520.896557,120079,0.0,-21.0,-0.005845,882.884917,5.297310,0.375169,1459.418490,7.721111,275.429177,8.215769,0.581862,2555.759487


In [51]:
# Plot de cumulatieve kinetische energie en het energieverlies door rolweerstand tegen de afgelegde afstand
plt.figure(figsize=(10, 6))

# Plot de kinetische energie zonder weerstand
plt.plot(x, df_e['Kinetische Energie Totaal (J)'], label='Kinetische Energie theoretisch (berekend)', color='g')
plt.plot(x, df_e['Kinetische Energie Totaal (J)']-df_e['Cumulatieve Energieverlies door rolweerstand (J)'], label='Kinetische Energie in situatie met rolweerstand (berekend)', color='b')
plt.plot(x, df_e['Kinetische Energie Totaal (J)']-df_e['Cumulatieve Energieverlies door rolweerstand (J)']-df_e['Cumulatieve Energieverlies door luchtweerstand (J)'], label='Kinetische Energie in situatie met rolweerstand en luchtweerstand (berekend)', color='y')
plt.scatter(x, df_e_kin['Kinetische Energie (J)'], label='Kinetische Energie (Berekend uit gemeten snelheid)', color='r')

# Labels en titel toevoegen
plt.xlabel('Afgelegde Afstand (m)')
plt.ylabel('Energie (J)')
plt.title('Kinetische Energie en Energieverlies door Rolweerstand over Afgelegde Afstand')
plt.legend()
plt.grid(True)

# Toon de grafiek
plt.show()
